## Exercise sheet \#3

Here, you will have to implement a web scraping engine, which will retrieve data from the famous [Internet Movie DataBase](https://www.imdb.com/) (IMDB).
The data we are interested in is **movie ratings**. 

Before starting, we need to keep in mind that requesting webpage could be time consuming. Should we want to build a large dataset, we would need to pay attention to the number of requests sent by our scraping engine (and to their frequency to avoid being black listed by IMDB's servers).

### Question 1

Let us inspect the following IMDB entry : [Movie tt0211915](https://www.imdb.com/title/tt0211915/).

On this page, you can find at least two ratings: the one by IMDB and the one by mediacritic. By pressing Ctrl+U, you can access the page's source code, and on chrome by clicking F12 you can launch the DevTools panel.

Where are located these ratings in the underlying HTML code ?

_To be completed_

### Question 2
Write a python program which retrieves the page located at the URL given above, parses it and extract the ratings of that movie.

In [31]:
import requests
page=requests.get('https://www.imdb.com/title/tt0211915/')

from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, 'html.parser')
# print(soup.prettify())
metacritic = soup.find_all(class_='metacriticScore score_favorable titleReviewBarSubItem')[0]
score = metacritic.get_text()

reviewer = soup.find_all(class_='ratingValue')[0]
score2 = reviewer.get_text()
print(score, score2)


69
 
8.3/10 


### Question 3

By using IMDB's advanced search feature, can you list the movies released in 2018 by decreasing number of ratings (not descreasing ratings!) ? Describes the actions you performed on IMDB's web interface.

_To be completed_

The URL of the first page of the search results looks like the following (250 results per page to save time during future retrieval):

`https://www.imdb.com/search/title?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&count=250`

Note also the URL of the second page of the search results:

`https://www.imdb.com/search/title?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&count=250&start=251&ref_=adv_nxt`

### Question 4

Write a python program which retrieves the ten first pages of results and store them locally (that is, save a local copy of each of them). Apply a 3-second sleep between requests.

In [43]:
import scrapy

class QuotesSpider(scrapy.Spider):
    name = 'MovieSpider'
    start_urls = ['https://www.imdb.com/search/title?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&count=250']
       
    def parse(self, response):
        page = response.url.split('/')
        filename = 'movie_search-%s.html' %page
        with open(filename, 'wb') as f:
            f.write(response.body)
        next_page = response.css('a.lister-page-next\ next-page::attr("href")').extract_first()
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)


if name=='main':
    import scrapy.crawler
    myspider = QuotesSpider()
    process = scrapy.crawler.CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
    process.crawl(myspider)
    process.start()

ModuleNotFoundError: No module named 'scrapy'

### Question 5
Describes the path where the movie information appears in the HTML tree of the retrieved webpages. 

_To be completed_

### Question 6
Extend the python program from Question 4 so that the following information is extracted for each movie:

- name of the movie
- year of release
- number of votes
- IMDB rating
- Metacritic score


In [ ]:
    def parse(self, response):
        with open('movies.txt', ‘a’) as f:
            movie = response.html(“div.quote”)
            for quote in quotes:
               text = quote.css(‘span.text::text’).extract_first()
               print(text,file=f)
            next_page = response.css(‘li.next a::attr(href)’).extract_first()
    if next_page is not None:
        yield response.follow(next_page, callback=self.parse)


### Question 7
Extend the python program from Question 6 so that the results of extraction are stored in a [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values).

### Question 8
Manually inspect the results of your data extraction. Are there any errors (missing information, ill-formed data, etc.) ? If so, extend your python script to fix these.

## References
On-line [tutorial](https://www.dataquest.io/blog/web-scraping-beautifulsoup/) entitled "Web Scraping with Python and BeautifulSoup" by Alex Olteanu.